<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import sys
import random
import copy
import string
import time
import datetime
from tqdm import tqdm
from scipy.stats import pareto
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rng = np.random.default_rng(int(time.time()))

In [2]:
exec(open('../utils.py').read())

In [3]:
exec(open('../utils-extended.py').read())

In [4]:
SAME_G = True

DEBUG = False
def print_debug(s):
    if DEBUG: print(s)


In [5]:
exec(open('../algorithms.py').read())

In [6]:
def generator(beta, n, m, T, p, alpha, f, delta):
    n_f = int(n*f)

    lat_util = np.zeros((n,3))

    # Initialize utilities
    rand_bits = (rng.random(n) < p)
    
    items = pareto.rvs(delta, size=n) # other artists
    for i in range(n):
        jj = int(rng.random()*m)
        lat_util[i][jj] = items[i]
    
    # generate protected groups
    grp_men = set(rng.choice(n, n - n_f, replace=False))
    all = set([i for i in range(n)])
    grps = [grp_men, all.difference(grp_men)]

    # generate observed utility
    obs_util = copy.deepcopy(lat_util)
    for i in grps[1]:
        for j in range(m):
            obs_util[i][j] *= beta

    print_debug((obs_util == lat_util).all())
    
    return lat_util, obs_util, grps


def get_details_top_k(sol, k):
    other_artists = set([i+n_alpha for i in range(n-n_alpha)])
    other_artists = other_artists.intersection(sol)
    other_artists_utils  = marg_F_mult(set(), other_artists, lat_util, m)
    top_5_other_artists = get_top_n(other_artists, other_artists_utils, k)
    print("other: ")
    for i in top_5_other_artists:
        print_details(i, lat_util, obs_util)

    emerging_artists = set([i for i in range(n_alpha)])
    emerging_artists = emerging_artists.intersection(sol)
    emerging_artists_utils  = marg_F_mult(set(), emerging_artists, lat_util, m)
    top_5_emerging_artists = get_top_n(emerging_artists, emerging_artists_utils, k)
    print('')
    print("emerg: ")
    for i in top_5_emerging_artists:
        print_details(i, lat_util, obs_util)

In [7]:
baseline_uncons = baseline_greedy_uncons
algo_disj = algo_3_disjoint_attr

algorithms = [baseline_uncons, algo_disj
colors = ['red', 'blue']
names = ['Uncons', 'Algorithm 1'] 
styles = ['-', '-'] 

# parameters
ITER = -1; n = 0; k = 0; m = 0
T = 0; p = 0; alpha = -1
f_list = []
delta_list = []

f = -1; delta = -1

SAME_G = True
func = lambda x: np.log(1 + x) 
func2 = lambda x: np.log(1 + x)


def run(T=1, p=0.9, alpha=0.0, f=0.5, delta=1):
    SAME_G = True
    func = lambda x: np.log(1 + x) 
    func2 = lambda x: np.log(1 + x)
    
    # T: max plays of emerging artist
    # p: probability new song (seems robust to the choice of p)
    # alpha: fraction of emerging artists

    # parameters
    ITER = 20
    n = 250
    k = 50
    m = 3

    DEBUG = False

    x=[]
    ys = [[] for i in range(len(algorithms))]
    yerrs = [[] for i in range(len(algorithms))]

    for beta in [0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95, 1]:
        lists = [[] for i in range(len(algorithms))]

        for ijk in range(ITER):
            lat_util, obs_util, grps = generator(beta, n, m, T, p, alpha, f, delta)
            n_alpha = int(n*alpha)
            n_f = int(n*f)
            
            sols = []
            utils = []
            
            # run algorithms and baselines
            sol_opt = baseline_uncons(lat_util, grps, k, m)
            util_opt = F(sol_opt, lat_util, m)
            for algo in algorithms: sols.append(algo(obs_util, grps, k, m))
            for s in sols: utils.append(F(s, lat_util, m))
            
            for i, u in enumerate(utils): lists[i].append(u / util_opt)

        for i, u in enumerate(lists): ys[i].append(np.mean(lists[i]))
        for i, u in enumerate(lists): yerrs[i].append(np.std(lists[i]) / np.sqrt(ITER))
        
        x.append(beta)

    fig, ax = plt.subplots()
    x = np.array(x)
    for i, u in enumerate(lists):
        plt.errorbar(x, ys[i], yerr=yerrs[i], linestyle=styles[i],color=colors[i], label=names[i],  linewidth=8, alpha=1.0)
    
    plt.title(f'Simulation with parameters:'
             +f'n={n}, k={k}, frac.-of-non-men={f},SAME_G={SAME_G} (log(1+x))\n'
             +f'm={m}, power-law-dist (delta={delta}), ITER={ITER}, weight_F={np.round(weight_F,2)}\n\n', fontsize=18)
    plt.ylim(0.75, 1.01)
    ax.set_ylabel('Normalized Latent Utility',fontsize=32)
    ax.set_xlabel('$\\beta$', fontsize=34)
    legend = plt.legend(loc='best', shadow=False, fontsize=26)
    plt.tick_params(axis='both', which='major', labelsize=26)  
    
    print()
    for i, u in enumerate(lists): print(f'{names[i]}: {ys[i]}')
    for i, u in enumerate(lists): print(f'{names[i]}_err: {yerrs[i]}')
    
    # plt.show()
    pdf_savefig()


In [12]:
f_list = [0.25, 0.5, 0.75]
delta_list = [1.0, 2, 3.0]
weight_F = [1, 1, 1]

for f in f_list:
    for delta in tqdm(delta_list):
        run(T=1, p=0.9, alpha=0.0, f=f, delta=delta)
        

  0%|                                                     | 0/3 [00:00<?, ?it/s]


Uncons: [0.9196005991350843, 0.9636899560035121, 0.9815641699169703, 0.9900188340465299, 0.9956152012570948, 0.9988848505660366, 0.9995672204108287, 0.999919577838732, 0.9999862069013382, 1.0]
Algorithm 1: [0.9951077547338916, 0.9949093786962445, 0.9955155953307413, 0.9961183825104969, 0.9968377216413609, 0.9965303219149598, 0.9976842841941369, 0.9982083854216937, 0.997835034972411, 0.9974514495639436]
Uncons_err: [0.014149875225964401, 0.003056749002802954, 0.0022683686656580584, 0.001220624945199142, 0.0005478092331101649, 0.00018940270342882775, 9.625484988441315e-05, 2.0815971746148853e-05, 5.285439791548889e-06, 0.0]
Algorithm 1_err: [0.0007258657695884042, 0.0009609531148303691, 0.0010061627319496256, 0.0005623816339149009, 0.0005886758984834714, 0.0007737427257210899, 0.0003220208710564283, 0.0003804017480466851, 0.0005833209702991243, 0.00048015201573681427]
./figs/H21M49S21_07-19-23pmmcs.pdf


 33%|███████████████                              | 1/3 [01:17<02:35, 77.98s/it]


Uncons: [0.9340029374923011, 0.9441292585213056, 0.9482809129254376, 0.9631137686687449, 0.9834928266931307, 0.9945859542957851, 0.9979249194291437, 0.9994506944426271, 0.9998791005321259, 1.0]
Algorithm 1: [0.9942561056991066, 0.9945972355774952, 0.9946585323575133, 0.995664915297614, 0.9954749946374056, 0.9969529038600637, 0.9976300503212855, 0.9975205841184736, 0.9975805345008263, 0.9972737644293975]
Uncons_err: [0.00804504644426258, 0.005166304526882395, 0.003985186344953361, 0.0022569512352299636, 0.0010380280664554595, 0.00041225924802117266, 0.0002478251803092695, 8.131022494520739e-05, 2.3638740615510112e-05, 0.0]
Algorithm 1_err: [0.0011662347632730003, 0.0010593673018788913, 0.0009416627073572461, 0.0007210640009031435, 0.0009357420185826012, 0.0005374571842005947, 0.0003971583914128529, 0.0004978690898751213, 0.0005589009035450738, 0.00033922417192715584]
./figs/H21M50S38_07-19-23genxf.pdf


 67%|██████████████████████████████               | 2/3 [02:35<01:17, 77.78s/it]

...


Uncons: [0.8482888933226883, 0.9495900674279916, 0.9723788795305468, 0.9802358509246135, 0.9941425562793009, 0.9984316943255169, 0.9994171683883035, 0.9999018243113129, 0.9999524224482581, 1.0]
Algorithm 1: [0.9936712061727349, 0.9966452409485473, 0.9935456179995338, 0.995401030854911, 0.9967604573771645, 0.995845996322495, 0.9939988187989751, 0.9913180104426267, 0.9959196423373177, 0.996324824396134]
Uncons_err: [0.017848207284139734, 0.00622076616001449, 0.0033435245080675525, 0.0024148637870093663, 0.0006557782592387556, 0.00020903692546511948, 9.156029524975585e-05, 2.322996183959127e-05, 1.1898190597550016e-05, 0.0]
Algorithm 1_err: [0.0013309897794741585, 0.0004970824540985392, 0.0011444244209114771, 0.0006539832227313781, 0.0005924115748340114, 0.0006940295366568655, 0.001258909625023781, 0.0015509972025598239, 0.00049551428610844, 0.0004877614562288235]
./figs/H21M53S19_07-19-23yxwll.pdf


 33%|██████████████▋                             | 1/3 [01:51<03:42, 111.04s/it]


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x12935fd30> (for post_execute):



KeyboardInterrupt

